In [1]:
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using Parameters
using StaticArrays
using ProfileView

using GridInterpolations 
using POMDPs
using POMDPToolbox
using QMDP
using JLD


In [2]:
include("../src/pedestrian_crossing/pomdp_types.jl")
include("../src/pedestrian_crossing/spaces.jl")
include("../src/pedestrian_crossing/transition.jl")
include("../src/pedestrian_crossing/observation.jl")
include("../src/pedestrian_crossing/belief.jl")
include("../src/pedestrian_crossing/policy.jl")
include("../src/pedestrian_crossing/frenet_pedestrian_pomdp.jl")
include("../src/pedestrian_crossing/helpers.jl")

animate_record (generic function with 2 methods)

In [3]:
pomdp = SingleOCFPOMDP()
println(n_states(pomdp))
solver = QMDPSolver(max_iterations=1, tolerance=1e-3, verbose=true) 
@requirements_info solver pomdp

# run the solver
qmdp_policy = solve(solver, pomdp)

JLD.save("policy.jld", "policy", qmdp_policy)

107251


Progress:   0%|                                         |  ETA: 8:12:32


INFO: POMDPs.jl requirements for solve(::QMDPSolver, ::POMDPs.POMDP) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::QMDPSolver, ::POMDPs.POMDP):
  [No additional requirements]
For solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP}) (in solve(::QMDPSolver, ::POMDPs.POMDP)):
  [✔] discount(::SingleOCFPOMDP)
  [✔] n_states(::SingleOCFPOMDP)
  [✔] n_actions(::SingleOCFPOMDP)
  [✔] transition(::SingleOCFPOMDP, ::SingleOCFState, ::SingleOCFAction)
  [✔] reward(::SingleOCFPOMDP, ::SingleOCFState, ::SingleOCFAction, ::SingleOCFState)
  [✔] state_index(::SingleOCFPOMDP, ::SingleOCFState)
  [✔] action_index(::SingleOCFPOMDP, ::SingleOCFAction)
  [✔] actions(::SingleOCFPOMDP, ::SingleOCFState)
  [✔] iterator(::Array)
  [✔] iterator(::Array)
  [✔] iterator(::SparseCat)
  [✔] pdf(::SparseCat, ::SingleOCFState)
For ordered_states(::Union{POMDPs.MDP,POMDPs.POMDP}) (in solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP})):
  [✔] states(::SingleOCFPO

Progress: 100%|█████████████████████████████████████████|  ETA: 0:00:00

[Iteration 1   ] residual:        348 | iteration runtime:  

Progress: 100%|█████████████████████████████████████████| Time: 0:01:18


78088.391 ms, (      78.1 s total)


In [6]:
pomdp = SingleOCFPOMDP()

qmdp_policy = load("policy.jld")["policy"];
println(fieldnames(qmdp_policy))
qmdp_policy.action_map

b = SparseCat([SingleOCFState(1.0, 19.0, 30.0, -5.0, 1.57, 0.0)],[1.])

action(qmdp_policy,b)

Symbol[:pomdp, :alphas, :action_map]


2-element SingleOCFAction:
 0.0
 0.0

ERROR (unhandled task failure): InterruptException:
Stacktrace:
 [1] process_events at ./libuv.jl:82 [inlined]
 [2] wait() at ./event.jl:216
 [3] wait(::Condition) at ./event.jl:27
 [4] stream_wait(::Timer, ::Condition, ::Vararg{Condition,N} where N) at ./stream.jl:42
 [5] wait(::Timer) at ./event.jl:357
 [6] (::Base.##302#303{IJulia.#send_stderr,Timer})() at ./event.jl:430


In [51]:
#=
1.07692
  2.15385
  3.23077
  4.30769
  5.38462
  6.46154
  7.53846
  8.61538
=#

b = SparseCat([SingleOCFState(-1.0, 14.0, 4.0, 1.0, 1.57, 0.0)],[1.])
act = action(qmdp_policy, b) # policy

#s = SingleOCFState(0.0, 4.3076923076923075, 40.0, 4.0, 1.57, 1.5)
#sp = SingleOCFState(0.0, 4.3076923076923075, 40.0, 4.0, 1.57, 1.5)

#act = SingleOCFAction(-4.0, 0.0)
#reward(pomdp, s, act, sp) 


SingleOCFAction(0.0, -1.0)

In [4]:
using Interact
using Plots
gr()

policy_grid = Matrix(length(pomdp.T_RANGE),length(pomdp.S_RANGE))

@manipulate for ego_v in pomdp.EGO_V_RANGE
    @manipulate for ped_v in pomdp.PED_V_RANGE

    #ego_v = 5.38462
    for i=1:length(pomdp.T_RANGE)
        ped_t = pomdp.T_RANGE[i]
        for j=1:length(pomdp.S_RANGE)
            ped_s = pomdp.S_RANGE[j]
            b = SparseCat([SingleOCFState(0.0, ego_v, ped_s, ped_t, 1.57, ped_v)],[1.])
            act = action(qmdp_policy, b) 
          #  println(ped_s, "/", ped_t, " act: ", act.acc)
            policy_grid[i,j] = act.acc
        end
    end

    xs = [pomdp.S_RANGE[i] for i = 1:length(pomdp.S_RANGE)]
    ys = [pomdp.T_RANGE[i] for i = 1:length(pomdp.T_RANGE)]
    heatmap(xs,ys,policy_grid,aspect_ratio=1)


    end
end




HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e00ed49da8de65dda19a6236dc3d0d6a7bfde9fd-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e00ed49da8de65dda19a6236dc3d0d6a7bfde9fd-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #Node#2(::Array{Any,1}, ::Type{T} where T, ::Symbol) at ./deprecated.jl:57
 [3] (::Core.#kw#Type)(::Array{Any,1}, ::Type{WebIO.Node}, ::Symbol) at ./<missing>:0
 [4] #input#34(::WebIO.JSString, ::Array{Any,1}, ::Void, ::String, ::String, ::String, ::Dict{Any,Any}, ::Observables.Observable{Int64}, ::Bool, ::WebIO.JSString, ::Dict{Any,Any}, ::String, ::String, ::Array{Any,1}, ::Widgets.#input, ::InteractBulma.Bulma, ::Observables.Observable{Float64}) at /home/xubuntu/.julia/v0.6/InteractBase/src/input.jl:169
 [5] (::Widgets.#kw##input)(::Array{Any,1}, ::Widgets.#input, ::InteractBulma.Bulma, ::Observables.Observable{Float64}) at ./<missing>:0
 [6] #input#163(::Array{Any,1}, ::Function, ::Observables.Observable{Float64}, ::Vararg{Observables.Observable{Float64},N} where N) at /home/xubuntu/.julia/v0.6/InteractBase/src/defaults.jl:12
 [7] (::Widgets.#kw##input)(::Array{Any,1}, ::Widgets.#input, ::Observables.Observable

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #Node#2(::Array{Any,1}, ::Type{T} where T, ::Symbol) at ./deprecated.jl:57
 [3] (::Core.#kw#Type)(::Array{Any,1}, ::Type{WebIO.Node}, ::Symbol) at ./<missing>:0
 [4] #slider#63(::String, ::Bool, ::Bool, ::Void, ::String, ::Observables.Observable{Float64}, ::Int64, ::Array{Any,1}, ::InteractBase.#slider, ::InteractBulma.Bulma, ::UnitRange{Int64}) at /home/xubuntu/.julia/v0.6/InteractBase/src/input.jl:321
 [5] (::InteractBase.#kw##slider)(::Array{Any,1}, ::InteractBase.#slider, ::InteractBulma.Bulma, ::UnitRange{Int64}) at ./<missing>:0
 [6] #slider#179(::Array{Any,1}, ::Function, ::UnitRange{Int64}, ::Vararg{UnitRange{Int64},N} where N) at /home/xubuntu/.julia/v0.6/InteractBase/src/defaults.jl:12
 [7] (::InteractBase.#kw##slider)(::Array{Any,1}, ::InteractBase.#slider, ::UnitRange{Int64}) at ./<missing>:0
 [8] #slider#68(::Float64, ::Array{Any,1}, ::Function, ::InteractBulma.Bulma, ::Array{Float64,1}) at /home/xubun

(div
  Widgets.Widget{:slider}(DataStructures.OrderedDict{Symbol,Any}(:changes=>Observables.Observable{Int64}("ob_03", 0, Any[WebIO.SyncCallback(WebIO.Scope("knockout-component-6edddf19-2a76-443f-a19c-63ae088e815e", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["ego_v"], Dict{Symbol,Any}(Pair{Symbol,Any}(:className, "interact "),Pair{Symbol,Any}(:style, Dict{Any,Any}(Pair{Any,Any}(:padding, "5px 10px 0px 10px")))), 1)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"text-align:right;width:18%"))), 2), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(Pair{Symbol,Any}(:max, 15),Pair{Symbol,Any}(:min, 1),Pair{Symbol,Any}(:attributes, Dict{Any,Any}(Pair{Any,Any}(:type, "range"),Pair{Any,Any}(Symbol("data-bind"), "numericValue: internalvalue, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}"))),Pair{Symbol,Any}(:step, 1),Pair{Symbol,Any}(:className, "slider slider is-fullwidth"),Pair{Symbol,Any}(:style, Dict{Any,Any}())), 0)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"flex-grow:1; margin: 0 2%"))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("data-bind"=>"text: displayedvalue"))), 0)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"width:18%"))), 1)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"display:flex; justify-content:center; align-items:center;"))), 7), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("changes", (Observables.Observable{Int64}(#= circular reference @-7 =#), nothing)),Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("internalvalue", (Observables.Observable{Int64}("ob_02", 8, Any[WebIO.SyncCallback(WebIO.Scope(#= circular reference @-7 =#), WebIO.SyncCallback(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37))]), nothing)),Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("value", (Observables.Observable{Float64}("ob_01", 7.0, Any[WebIO.SyncCallback(WebIO.Scope(#= circular reference @-7 =#), WebIO.SyncCallback(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37)), Observables.g]), nothing))), Set{String}(), nothing, Any["knockout"=>"/home/xubuntu/.julia/v0.6/Knockout/src/../assets/knockout.js", "knockout_punches"=>"/home/xubuntu/.julia/v0.6/Knockout/src/../assets/knockout_punches.js", "/home/xubuntu/.julia/v0.6/InteractBase/src/../assets/all.js", "/home/xubuntu/.julia/v0.6/InteractBase/src/../assets/style.css", "/home/xubuntu/.julia/v0.6/InteractBulma/src/../assets/main.css"], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"internalvalue\\\":8,\\\"value\\\":7.0}

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #Node#2(::Array{Any,1}, ::Type{T} where T, ::Symbol, ::Widgets.Widget{:slider}, ::Vararg{Any,N} where N) at ./deprecated.jl:57
 [3] WebIO.Node(::Symbol, ::Widgets.Widget{:slider}, ::Observables.Observable{Any}) at ./deprecated.jl:57
 [4] (::##32#38)(::Widgets.Widget{:manipulate}) at /home/xubuntu/.julia/v0.6/InteractBase/src/manipulate.jl:92
 [5] render at /home/xubuntu/.julia/v0.6/InteractBase/src/widget.jl:3 [inlined]
 [6] show at /home/xubuntu/.julia/v0.6/InteractBase/src/widget.jl:5 [inlined]
 [7] limitstringmime(::MIME{Symbol("text/plain")}, ::Widgets.Widget{:manipulate}) at /home/xubuntu/.julia/v0.6/IJulia/src/inline.jl:36
 [8] display_dict(::Widgets.Widget{:manipulate}) at /home/xubuntu/.julia/v0.6/IJulia/src/execute_request.jl:28
 [9] (::Compat.#inner#14{Array{Any,1},IJulia.#display_dict,Tuple{Widgets.Widget{:manipulate}}})() at /home/xubuntu/.julia/v0.6/Compat/src/Compat.jl:332
 [10] execute_request(::ZMQ.

In [6]:
using Interact
using Plots
gr()

policy_grid = Matrix(length(pomdp.T_RANGE),length(pomdp.S_RANGE))

@manipulate for ego_v in pomdp.EGO_V_RANGE
    @manipulate for ped_v in pomdp.PED_V_RANGE

    #ego_v = 5.38462
    for i=1:length(pomdp.T_RANGE)
        ped_t = pomdp.T_RANGE[i]
        for j=1:length(pomdp.S_RANGE)
            ped_s = pomdp.S_RANGE[j]
            b = SparseCat([SingleOCFState(0.0, ego_v, ped_s, ped_t, 1.57, ped_v)],[1.])
            act = action(qmdp_policy, b) 
          #  println(ped_s, "/", ped_t, " act: ", act.acc)
            policy_grid[i,j] = act.lateral_movement
        end
    end

    xs = [pomdp.S_RANGE[i] for i = 1:length(pomdp.S_RANGE)]
    ys = [pomdp.T_RANGE[i] for i = 1:length(pomdp.T_RANGE)]
    heatmap(xs,ys,policy_grid,aspect_ratio=1)


    end
end


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #Node#2(::Array{Any,1}, ::Type{T} where T, ::Symbol) at ./deprecated.jl:57
 [3] (::Core.#kw#Type)(::Array{Any,1}, ::Type{WebIO.Node}, ::Symbol) at ./<missing>:0
 [4] #input#34(::WebIO.JSString, ::Array{Any,1}, ::Void, ::String, ::String, ::String, ::Dict{Any,Any}, ::Observables.Observable{Int64}, ::Bool, ::WebIO.JSString, ::Dict{Any,Any}, ::String, ::String, ::Array{Any,1}, ::Widgets.#input, ::InteractBulma.Bulma, ::Observables.Observable{Float64}) at /home/xubuntu/.julia/v0.6/InteractBase/src/input.jl:169
 [5] (::Widgets.#kw##input)(::Array{Any,1}, ::Widgets.#input, ::InteractBulma.Bulma, ::Observables.Observable{Float64}) at ./<missing>:0
 [6] #input#163(::Array{Any,1}, ::Function, ::Observables.Observable{Float64}, ::Vararg{Observables.Observable{Float64},N} where N) at /home/xubuntu/.julia/v0.6/InteractBase/src/defaults.jl:12
 [7] (::Widgets.#kw##input)(::Array{Any,1}, ::Widgets.#input, ::Observables.Observable

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #Node#2(::Array{Any,1}, ::Type{T} where T, ::Symbol, ::String, ::Vararg{String,N} where N) at ./deprecated.jl:57
 [3] (::Core.#kw#Type)(::Array{Any,1}, ::Type{WebIO.Node}, ::Symbol, ::String) at ./<missing>:0
 [4] #wdglabel#73(::Int64, ::Int64, ::Int64, ::Int64, ::String, ::Dict{Any,Any}, ::Array{Any,1}, ::Function, ::InteractBulma.Bulma, ::String) at /home/xubuntu/.julia/v0.6/InteractBase/src/input.jl:350
 [5] wdglabel(::InteractBulma.Bulma, ::String) at /home/xubuntu/.julia/v0.6/InteractBase/src/input.jl:348
 [6] #wdglabel#201(::Array{Any,1}, ::Function, ::String, ::Vararg{String,N} where N) at /home/xubuntu/.julia/v0.6/InteractBase/src/defaults.jl:12
 [7] #slider#63(::String, ::Bool, ::Bool, ::Void, ::String, ::Observables.Observable{Float64}, ::Int64, ::Array{Any,1}, ::InteractBase.#slider, ::InteractBulma.Bulma, ::UnitRange{Int64}) at /home/xubuntu/.julia/v0.6/InteractBase/src/input.jl:321
 [8] (::InteractBase

(div
  Widgets.Widget{:slider}(DataStructures.OrderedDict{Symbol,Any}(:changes=>Observables.Observable{Int64}("ob_35", 0, Any[WebIO.SyncCallback(WebIO.Scope("knockout-component-df80462b-f089-4c5c-84ab-2e3936090a02", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["ego_v"], Dict{Symbol,Any}(Pair{Symbol,Any}(:className, "interact "),Pair{Symbol,Any}(:style, Dict{Any,Any}(Pair{Any,Any}(:padding, "5px 10px 0px 10px")))), 1)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"text-align:right;width:18%"))), 2), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(Pair{Symbol,Any}(:max, 15),Pair{Symbol,Any}(:min, 1),Pair{Symbol,Any}(:attributes, Dict{Any,Any}(Pair{Any,Any}(:type, "range"),Pair{Any,Any}(Symbol("data-bind"), "numericValue: internalvalue, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}"))),Pair{Symbol,Any}(:step, 1),Pair{Symbol,Any}(:className, "slider slider is-fullwidth"),Pair{Symbol,Any}(:style, Dict{Any,Any}())), 0)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"flex-grow:1; margin: 0 2%"))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("data-bind"=>"text: displayedvalue"))), 0)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"width:18%"))), 1)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"display:flex; justify-content:center; align-items:center;"))), 7), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("changes", (Observables.Observable{Int64}(#= circular reference @-7 =#), nothing)),Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("internalvalue", (Observables.Observable{Int64}("ob_34", 8, Any[WebIO.SyncCallback(WebIO.Scope(#= circular reference @-7 =#), WebIO.SyncCallback(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37))]), nothing)),Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("value", (Observables.Observable{Float64}("ob_33", 7.0, Any[WebIO.SyncCallback(WebIO.Scope(#= circular reference @-7 =#), WebIO.SyncCallback(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37)), Observables.g]), nothing))), Set{String}(), nothing, Any["knockout"=>"/home/xubuntu/.julia/v0.6/Knockout/src/../assets/knockout.js", "knockout_punches"=>"/home/xubuntu/.julia/v0.6/Knockout/src/../assets/knockout_punches.js", "/home/xubuntu/.julia/v0.6/InteractBase/src/../assets/all.js", "/home/xubuntu/.julia/v0.6/InteractBase/src/../assets/style.css", "/home/xubuntu/.julia/v0.6/InteractBulma/src/../assets/main.css"], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"internalvalue\\\":8,\\\"value\\\":7.0}

ERROR (unhandled task failure): InterruptException:
Stacktrace:
 [1] process_events at ./libuv.jl:82 [inlined]
 [2] wait() at ./event.jl:216
 [3] wait(::Condition) at ./event.jl:27
 [4] stream_wait(::Timer, ::Condition, ::Vararg{Condition,N} where N) at ./stream.jl:42
 [5] wait(::Timer) at ./event.jl:357
 [6] (::Base.##302#303{IJulia.#send_stderr,Timer})() at ./event.jl:430


In [44]:
# Test action space
action_space = actions(pomdp)
n_actions(pomdp)
x = action_space[action_index(pomdp,SingleOCFAction(-4.0, 0.0) )]
println(x)

cnt = 0
for (index, a) in enumerate(pomdp.action_space)
    idx = action_index(pomdp,a)
    if (idx != index)
        println("error")
        cnt = cnt + 1
    end
    println(a)
end
println(cnt)


SingleOCFAction(-4.0, 0.0)
SingleOCFAction(1.0, 0.0)
SingleOCFAction(0.0, 0.0)
SingleOCFAction(-1.0, 0.0)
SingleOCFAction(-2.0, 0.0)
SingleOCFAction(-4.0, 0.0)
0


In [50]:
state_space = states(pomdp)

state_space[state_index(pomdp,SingleOCFState(0.0, 4, 40.0, 0.0, 1.57, 1.5))]

SingleOCFState(0.0, 4.3076923076923075, 40.0, 0.0, 1.57, 1.5)

In [23]:

s = SingleOCFState(0.0, 4.3076923076923075, 40.0, 4.0, 1.57, 1.5)
sp = SingleOCFState(0.0, 4.3076923076923075, 40.0, 4.0, 1.57, 1.5)

act = SingleOCFAction(0.0, 1.0)
reward(pomdp, s, act, sp) 

-1.0

In [31]:
rng = MersenneTwister(1);

hr = HistoryRecorder(max_steps=100, rng=rng)
up = PreviousObservationUpdater() 
s0 = initial_state(pomdp, rng) 

b0 = initBeliefPedestrian(pomdp, s0)
updater = DiscreteUpdater(pomdp)

hist = simulate(hr, pomdp, qmdp_policy, up, b0, s0)

LoadError: [91mMethodError: no method matching POMDPToolbox.PreviousObservationUpdater()[39m

In [50]:
pomdp.EGO_V_RANGE

14-element Array{Float64,1}:
  0.0    
  1.07692
  2.15385
  3.23077
  4.30769
  5.38462
  6.46154
  7.53846
  8.61538
  9.69231
 10.7692 
 11.8462 
 12.9231 
 14.0    

3-element Array{Float64,1}:
 -1.0
  0.0
  1.0